FAIL: no me funcionó la apirest, no sé por qué. si que funcionó la librería python

In [ ]:
from utils_audio import tts_azure
tts_azure('Hola', 'testa.mp3',voice)
#     POST /cognitiveservices/v1 HTTP/1.1

# X-Microsoft-OutputFormat: riff-24khz-16bit-mono-pcm
# Content-Type: application/ssml+xml
# Host: westus.tts.speech.microsoft.com
# Content-Length: <Length>
# Authorization: Bearer [Base64 access_token]
# User-Agent: <Your application name>

# <speak version='1.0' xml:lang='en-US'><voice xml:lang='en-US' xml:gender='Male'
#     name='en-US-ChristopherNeural'>
#         I'm excited to try text to speech!
# </voice></speak>

import requests
import time
import base64

# Replace the below URL with your subscription key and service region (e.g., "westus").
# see https://portal.azure.com/#@milenkohalatgrupoavanza.onmicrosoft.com/resource/subscriptions/8358f1a6-fbca-47d8-ab61-b0d76eb44e02/resourceGroups/cloud-shell-storage-westeurope/providers/Microsoft.CognitiveServices/accounts/vozPrimera/overview
subscription_key = '5125c3bec5914db180595773dc409b63' # se encuentra en el portal de azure, en la cuenta de vozPrimera
region = 'westeurope'


def get_token(subscription_key):
    fetch_token_url = f'https://{region}.api.cognitive.microsoft.com/sts/v1.0/issueToken'
    headers = {
        'Ocp-Apim-Subscription-Key': subscription_key
    }
    response = requests.post(fetch_token_url, headers=headers)
    access_token = str(response.text)
    return access_token


def save_audio(text, filename, access_token, voice_name='es-ES-Pablo-Apollo'):
    access_token = get_token(subscription_key)
    base_url = f'https://{region}.tts.speech.microsoft.com/'
    path = 'cognitiveservices/v1'
    constructed_url = base_url + path

    headers = {
        'Authorization': 'Bearer ' + access_token,
        'Content-Type': 'application/ssml+xml',
        'X-Microsoft-OutputFormat': 'riff-24khz-16bit-mono-pcm',
        'User-Agent': 'YOUR_RESOURCE_NAME'
    }

    #post
    xml_body = f'<speak version="1.0" xmlns="http://www.w3.org/2001/10/synthesis" xml:lang="en-US"><voice name="{voice_name}">{text}</voice></speak>'
    response = requests.post(constructed_url, headers=headers, data=xml_body)
    print(f'\n **construct_url: {constructed_url}**')
    print(f'\n **headers: {headers}**')
    print(f'\n **xml_body: {xml_body}**')
    print(f'\n **response: {response.text}**')


    if response.status_code == 200:
        with open(filename, 'wb') as audio:
            audio.write(response.content)
            print(f'Audio written to {filename}')
    #401
    elif response.status_code == 401:
        print("\nStatus code: " + str(response.status_code) +
              "\nPlease make sure that you are using the correct subscription key.")
    #403
    elif response.status_code == 403:
        print("\nStatus code: " + str(response.status_code) +
              "\nPlease make sure you are subscribed to the speech service and region is set correctly.")
    else:
        print("\nStatus code: " + str(response.status_code) +
              "\nSomething went wrong. Check your subscription key and headers.\n")
        
    return response.status_code

access_token = get_token(subscription_key)
save_audio('Hola, esto es una prueba', 'test.mp3', access_token)